In [ ]:
pip install split-folders

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile
# import splitfolders
import os
import tensorflow as tf

In [3]:
class myCallback(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        if (logs.get('accuracy') > 0.85 and logs.get('val_accuracy') > 0.85):
            self.model.stop_training = True

In [4]:
# Extract the archive
local_zip = '/content/drive/MyDrive/dataset_capstone/dataset_capstone/archive_2.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('data/')
zip_ref.close()

In [5]:
# delete class
!rm -rf '/content/data/FaceShape Dataset/testing_set/Oval'
!rm -rf '/content/data/FaceShape Dataset/testing_set/Oblong'
!rm -rf '/content/data/FaceShape Dataset/training_set/Oval'
!rm -rf '/content/data/FaceShape Dataset/training_set/Oblong'

In [ ]:
# base_dir = 'data/dataset_capstone/'
# splitfolders.ratio(base_dir, output = 'data/dataset_capstone/dataset', seed = 1337, ratio=(0.8, 0.2), group_prefix=None)
# train_dir = 'data/dataset_capstone/dataset/train'
# val_dir = 'data/dataset_capstone/dataset/val'

In [6]:
train_dir = '/content/data/FaceShape Dataset/training_set'
val_dir = '/content/data/FaceShape Dataset/testing_set'

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

training_datagen = ImageDataGenerator(
      rescale = 1./255,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='wrap')

validation_datagen = ImageDataGenerator(
                    rescale=1./255)

In [8]:
train_generator = training_datagen.flow_from_directory(
	train_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

validation_generator = validation_datagen.flow_from_directory(
	val_dir,
	target_size=(224,224),
	class_mode='categorical',
  batch_size=32
)

Found 2400 images belonging to 3 classes.
Found 600 images belonging to 3 classes.


In [9]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Conv2D(256, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),

    tf.keras.layers.Flatten(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(3, activation='softmax')
])

In [28]:
# model.add(tf.keras.layers.BatchNormalization())

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 64)      1792      
                                                                 
 max_pooling2d (MaxPooling2  (None, 111, 111, 64)      0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 64)      36928     
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 52, 52, 128)       73856     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 26, 26, 128)       0

In [11]:
model.compile(loss = 'categorical_crossentropy', optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001), metrics=['accuracy'])

In [31]:
# model.compile(loss='categorical_crossentropy', optimizer=tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.9), metrics=['accuracy'])

In [12]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

In [14]:
# Train the model
callback = myCallback()
history = model.fit(train_generator, epochs=100, validation_data = validation_generator, verbose = 1, callbacks=callback)

Epoch 1/100
75/75 [==============================] - 46s 614ms/step - loss: 0.5869 - accuracy: 0.7608 - val_loss: 0.8748 - val_accuracy: 0.6367
Epoch 2/100
75/75 [==============================] - 45s 599ms/step - loss: 0.5846 - accuracy: 0.7575 - val_loss: 0.7919 - val_accuracy: 0.6600
Epoch 3/100
75/75 [==============================] - 45s 604ms/step - loss: 0.5976 - accuracy: 0.7396 - val_loss: 0.8794 - val_accuracy: 0.6317
Epoch 4/100
75/75 [==============================] - 45s 604ms/step - loss: 0.5931 - accuracy: 0.7550 - val_loss: 0.8083 - val_accuracy: 0.6650
Epoch 5/100
75/75 [==============================] - 47s 626ms/step - loss: 0.5647 - accuracy: 0.7658 - val_loss: 0.8302 - val_accuracy: 0.6600
Epoch 6/100
75/75 [==============================] - 46s 614ms/step - loss: 0.5729 - accuracy: 0.7633 - val_loss: 0.8561 - val_accuracy: 0.6467
Epoch 7/100
75/75 [==============================] - 46s 609ms/step - loss: 0.5760 - accuracy: 0.7550 - val_loss: 0.8596 - val_accuracy:

In [15]:
pip install tensorflowjs

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.2/89.2 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.3/15.3 MB 50.9 MB/s eta 0:00:00


In [16]:
import tensorflowjs

In [17]:
import time
saved_model_path = "./{}.h5".format(int(time.time()))

model.save(saved_model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [18]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./

2023-12-15 08:47:52.968538: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-12-15 08:47:52.968588: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-12-15 08:47:52.970463: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-12-15 08:47:54.998686: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [19]:
saved_model_path = "./{}.h5".format(int(time.time()))

history.save(saved_model_path)

AttributeError: ignored

In [ ]:
!tensorflowjs_converter --input_format=keras {saved_model_path} ./